  <picture>
  <source srcset="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes-w.svg" media="(prefers-color-scheme: dark)">
    <img src="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes.svg">
  </picture>

Veri(fy)log
===

In previous posts, we discussed Racket and Rosette. This is the post where we are going to verify our hardware designs using the infrastructure we have developed so far. We will be looking at verifying Verilog hardware designs with Rosette. Later in this tutorial, we will be validating a simple design based on Verilog implemented picoRV processor that works with a simple firmware that we implemented in one of my previous blog posts.

We need to follow several steps to convert Verilog designs to Racket (.rkt) native formats. For this purpose, we need [Yosys]() and [rtlv]().



In [1]:
#lang iracket/lang

(require rosette/safe)
(require racket)
(require racket/vector)

```shell
hello
```

In [2]:
(define (add-vectors a b)
  (define (full-adder x y c-in)
    (let* ((sum (bitwise-xor x y c-in))
           (carry (bitwise-ior (bitwise-and x y) (bitwise-and c-in (bitwise-xor x y)))))
      (cons sum carry)))
  (let* ((result (make-vector 32 0))
         (carry 0))
    (for ([i (in-range 31 -1 -1)])
      (let* ((full-add (full-adder (vector-ref a i) (vector-ref b i) carry))
             (sum (car full-add))
             (carry (cdr full-add)))
        (vector-set! result i sum)))
    result))

In [3]:
(add-vectors #(0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1)
               #(1 0 1 1 0 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 0 1 0 0 1))

'#(1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0)

In [12]:
(define (add-vectors2 vec1 vec2)
  (let loop ((carry 0)
             (result '())
             (vec1 (reverse vec1))
             (vec2 (reverse vec2)))
    (cond ((and (null? vec1) (null? vec2))
           (if (= carry 1)
               (cons 1 result)
               result))
          ((null? vec1)
           (loop carry (cons (+ carry (car vec2) 0) result) '() (cdr vec2)))
          ((null? vec2)
           (loop carry (cons (+ carry (car vec1) 0) result) (cdr vec1) '()))
          (else
           (let ((sum (+ (car vec1) (car vec2) carry)))
             (loop (if (> sum 1) 1 0)
                   (cons (modulo sum 2) result)
                   (cdr vec1)
                   (cdr vec2)))))))

In [14]:
(define vec1 (vector 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0))
(define vec2 (vector 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0))

(add-vectors vec1 vec2)


'#(0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)